In [ ]:
# Colab upload/download shims
try:
    from google.colab import files  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False
    class _FilesShim:
        def upload(self):
            print("Local mode: place input files in the expected folders; no upload dialog.")
            return {}
        def download(self, *args, **kwargs):
            print("Local mode: file saved on disk; no download dialog.")
    files = _FilesShim()

# Step 1: Single-image generative initialization (MoGE → 3DGS)

This notebook takes the first frame of a sequence, runs a monocular 3D generative model (MoGE-2), and converts the dense point cloud into a 3D Gaussian Splatting (3DGS) scene.

- Output: a dense initialization PLY for 3DGS with positions, SH-color (DC), opacity logits, scale and identity rotation.
- Use the produced PLY as the starting scene for the incremental on-the-fly pipeline (see Step 2).
- Works on Colab and locally if dependencies are available and paths are set appropriately.

Tips:
- Ensure your input image resides in your chosen scene directory.
- Verify the PLY attributes are in the expected 3DGS format before proceeding to Step 2.

In [ ]:
# Setup dependencies (Colab-friendly)
!pip install git+https://github.com/microsoft/MoGe.git plyfile opencv-python -q

import torch
import numpy as np
import cv2
from moge.model.v2 import MoGeModel
from plyfile import PlyData, PlyElement
import os

# --- Upload a single image ---
uploaded = files.upload()
if len(uploaded) == 0:
    raise RuntimeError("No image uploaded.")

# Get uploaded image filename
IMG_PATH = next(iter(uploaded))
print("Uploaded image:", IMG_PATH)

# --- Settings ---
OUTPUT_PLY = "moge_points.ply"
DOWNSAMPLE_MAX_PIXELS = 2_000_000  # prevent OOM if image is huge (optional)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --- Load model (MoGE-2) ---
print("Loading MoGE-2 (may take a while)...")
model = MoGeModel.from_pretrained("Ruicheng/moge-2-vitl-normal").to(device)
model.eval()
print("Model loaded.")

# --- Load and prepare image ---
if not os.path.exists(IMG_PATH):
    raise FileNotFoundError(f"Image not found: {IMG_PATH}")

bgr = cv2.imread(IMG_PATH)
if bgr is None:
    raise RuntimeError("cv2.imread returned None. Check the file path.")
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

H, W = rgb.shape[:2]
print("Image size:", W, "x", H)

# Optional: resize if image is extremely large
if H * W > DOWNSAMPLE_MAX_PIXELS:
    scale = (DOWNSAMPLE_MAX_PIXELS / (H*W)) ** 0.5
    newW, newH = max(1, int(W*scale)), max(1, int(H*scale))
    rgb = cv2.resize(rgb, (newW, newH), interpolation=cv2.INTER_AREA)
    print(f"Resized to {newW}x{newH} to avoid OOM.")
H, W = rgb.shape[:2]

# Convert to expected tensor: (3, H, W), values in [0,1]
input_tensor = torch.tensor(rgb / 255.0, dtype=torch.float32, device=device).permute(2, 0, 1)

# --- Inference ---
print("Running MoGE inference...")
with torch.no_grad():
    out = model.infer(input_tensor)

# Expected keys: "points" (H,W,3), "mask" (H,W), etc.
points = out.get("points")
mask = out.get("mask")

# Convert to numpy if needed
if isinstance(points, torch.Tensor):
    points = points.detach().cpu().numpy()
if isinstance(mask, torch.Tensor):
    mask = mask.detach().cpu().numpy()

if points is None or mask is None:
    raise RuntimeError("Model output missing 'points' or 'mask'.")

# --- Build list of valid points ---
valid_idx = np.where(mask.astype(bool))
ys, xs = valid_idx
pc_xyz = points[ys, xs, :]           # (N,3)
pc_colors = rgb[ys, xs, :]           # (N,3) uint8

print("Valid points:", pc_xyz.shape[0])

# --- Write a simple vertex-color PLY ---
N = pc_xyz.shape[0]
vertex_dtype = np.dtype([
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')
])

vertex_array = np.empty(N, dtype=vertex_dtype)
vertex_array['x'] = pc_xyz[:, 0].astype(np.float32)
vertex_array['y'] = pc_xyz[:, 1].astype(np.float32)
vertex_array['z'] = pc_xyz[:, 2].astype(np.float32)
vertex_array['red']   = pc_colors[:, 0].astype(np.uint8)
vertex_array['green'] = pc_colors[:, 1].astype(np.uint8)
vertex_array['blue']  = pc_colors[:, 2].astype(np.uint8)

el = PlyElement.describe(vertex_array, 'vertex')
PlyData([el]).write(OUTPUT_PLY)
print("Saved:", OUTPUT_PLY)

# --- Download the PLY (Colab only; local mode prints a message) ---
files.download(OUTPUT_PLY)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.8/709.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.1 MB/s eta 0:00:00


Saving frame-001469.color.jpg to frame-001469.color.jpg
Image uploadée : frame-001469.color.jpg
Device: cuda
Chargement du modèle MoGe-2 (peut prendre du temps)...


model.pt:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Modèle chargé.
Image size: 1296 x 968
Inférence MoGe...
Points valides extraits: 1254528
Saved: moge_points.ply


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Colab upload/download shims
try:
    from google.colab import files  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False
    class _FilesShim:
        def upload(self):
            print("Local mode: place input files in the expected folders; no upload dialog.")
        def download(self, *args, **kwargs):
            print("Local mode: file saved on disk; no download dialog.")
    files = _FilesShim()

In [ ]:
import numpy as np
from plyfile import PlyData, PlyElement
import io

print("--- Upload .ply (MoGE) ---")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")
input_filename = list(uploaded.keys())[0]
input_ply_bytes = uploaded[input_filename]
print("File:", input_filename)

# --- Read the MoGE-generated .ply ---
plydata = PlyData.read(io.BytesIO(input_ply_bytes))
vertices = plydata['vertex']
xyz = np.vstack([vertices['x'], vertices['y'], vertices['z']]).T  # positions (x,y,z)
r = np.asarray(vertices['red']).reshape(-1,1)
g = np.asarray(vertices['green']).reshape(-1,1)
b = np.asarray(vertices['blue']).reshape(-1,1)
srgb_colors = np.hstack((r,g,b)) / 255.0  # normalize colors to [0,1]
print(f"{len(xyz)} points read.")

colors = srgb_colors

# --- Compute SH-DC coefficients (features_dc) ---
C0 = 0.28209479177387814  # Y00 constant
# color reconstruction in renderer: color ≈ 0.5 + C0 * f_dc
# => f_dc = (color - 0.5) / C0
features_dc = (colors - 0.5) / C0

# safety clamp to avoid extreme outliers
features_dc = np.clip(features_dc, -5.0, 5.0)

# --- Opacity (logit for alpha ≈ 0.99) ---
alpha_target = 0.99
opacity_logit = np.log(alpha_target / (1.0 - alpha_target))  # logit(alpha)
opacities = opacity_logit * np.ones((len(xyz), 1), dtype=np.float32)

# --- Other attributes ---
scale_value = 0.01  # initial Gaussian size
scales = np.log(scale_value) * np.ones((len(xyz), 3), dtype=np.float32)  # log-space per axis
rotations = np.zeros((len(xyz), 4), dtype=np.float32)
rotations[:,0] = 1.0  # identity quaternion [1,0,0,0]
normals = np.zeros_like(xyz, dtype=np.float32)  # unused by 3DGS rasterization

# --- Info ---
print("srgb min/max:", srgb_colors.min(), srgb_colors.max())
print("colors min/max:", colors.min(), colors.max())
print("features_dc min/max:", features_dc.min(), features_dc.max())
print("opacity_logit:", opacity_logit)

# --- Create PLY (SuperSplat/GraphDeco layout) ---
dtype_full = [
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4'),
    ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
    ('opacity', 'f4'),
    ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
    ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4')
]

N = len(xyz)
elements = np.zeros(N, dtype=dtype_full)
attributes = np.hstack((xyz.astype(np.float32),
                        normals.astype(np.float32),
                        features_dc.astype(np.float32),
                        opacities.astype(np.float32),
                        scales.astype(np.float32),
                        rotations.astype(np.float32)))

# pack rows
elements[:] = [tuple(row) for row in attributes]

output_filename = "3dgs_SuperSplat.ply"
el = PlyElement.describe(elements, 'vertex')
PlyData([el]).write(output_filename)
print("Wrote:", output_filename)
files.download(output_filename)  # Colab only; local prints a message

In [ ]:
# SuperSplat → binary Little Endian (compatible with on-the-fly)
# Read the original PLY file
from plyfile import PlyData
ply = PlyData.read("3dgs_SuperSplat.ply")

# Save as binary by opening in binary write mode ('wb')
with open("3dgs_MoGe.ply", 'wb') as f:
    ply.write(f)

# Download the new file (Colab only; local prints a message)
files.download("3dgs_MoGe.ply")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>